In [4]:
!pip -qqq install pip --progress-bar off

!pip install -q torch datasets --progress-bar off
!pip install -q accelerate peft bitsandbytes trl --progress-bar off

!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip install rank_bm25


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.24.5 which is incompatible.
transformers 4.42.4 requires tokenizers<0.20,>=0.19, but you have tokenizers 0.15.2 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastembed 0.2.7 requires huggingface-hub<0.21,>=0.20, but you have huggingface-hub 0.24.5 which is incompatible.
fastembed 0.2.7 requires tokenizers<0.16,>=0.15, but you have tokenizers 0.19.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.20.0 requires huggingface-hu

In [6]:
!pip -qqq install langchain-groq==0.1.3 --progress-bar off

In [8]:

import os
import textwrap
from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse





from google.colab import drive
drive.mount('/content/drive')



Mounted at /content/drive


In [7]:
os.environ["GROQ_API_KEY"] = ""

# def print_response(response):

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))


In [ ]:
# Define the instruction for parsing documents, emphasizing precision, relevance, and neutrality.
instruction = """The provided documents collectively offer a comprehensive history of Nepal, spanning key periods, events, dynasties, and cultural developments. These documents include detailed accounts of various ruling periods, such as the Malla era, and the evolution of Nepal's infrastructure, society, and governance.

When answering questions based on these documents, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- When referencing specific periods or events, highlight relevant sections or passages where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific events or developments.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand various aspects of Nepal's history."""

# Initialize the LlamaParse object with API key, result type, parsing instructions, and timeout.
parser = LlamaParse(
    api_key = '',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)
# List of document paths to be parsed.
documents = [
    '/content/drive/My Drive/HistoryofNepal.pdf',
    '/content/drive/My Drive/AccountofNepal.pdf',
    '/content/drive/My Drive/royal_massacre.pdf',
    '/content/drive/My Drive/Maiost_Movement.pdf',
    '/content/drive/My Drive/History_of_Gorkha.pdf',
    '/content/drive/My Drive/History_of_Nepal_1.pdf'
]

# Loop through each document and parse it using the LlamaParse object.
for document_path in documents:
    llama_parse_document = await parser.aload_data(document_path)


In [ ]:
# Define the instruction for parsing documents, emphasizing precision, relevance, and neutrality.
instruction = """The provided documents collectively offer a comprehensive history of Nepal, spanning key periods, events, dynasties, and cultural developments. These documents include detailed accounts of various ruling periods, such as the Malla era, and the evolution of Nepal's infrastructure, society, and governance.

When answering questions based on these documents, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- When referencing specific periods or events, highlight relevant sections or passages where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific events or developments.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand various aspects of Nepal's history."""

# Initialize the LlamaParse object with API key, result type, parsing instructions, and timeout.
parser = LlamaParse(
    api_key = 'LLama_cloud_key',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)
# List of document paths to be parsed.
documents = [
    '/content/drive/My Drive/HistoryofNepal.pdf',
    '/content/drive/My Drive/AccountofNepal.pdf',
    '/content/drive/My Drive/royal_massacre.pdf',
    '/content/drive/My Drive/Maiost_Movement.pdf',
    '/content/drive/My Drive/History_of_Gorkha.pdf',
    '/content/drive/My Drive/History_of_Nepal_1.pdf'
]

# Loop through each document and parse it using the LlamaParse object.
for document_path in documents:
    llama_parse_document = await parser.aload_data(document_path)


In [9]:
# Extract the first parsed document from the list.
parsed_doc = llama_parse_document[0]

# View a snippet of the parsed document in Markdown format
Markdown(parsed_doc.text[1000:])

In [10]:
from pathlib import Path

# Define the path to save the parsed document as a Markdown file.
document_path = Path('/content/drive/MyDrive/all_doc_parsed_document.md')

# Open the Markdown file in append mode and write the parsed document's text to it.
with document_path.open('a') as f:
    f.write(parsed_doc.text)



### Vector embeddings

In [11]:
# Create a loader for the Markdown file to load its content.
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [12]:
# Split the loaded documents into chunks for further processing.
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 2048, chunk_overlap = 128)
documents = text_splitter.split_documents(loaded_documents)
print(documents[2].page_content)

At Home with Apartheid

Ginsburg, who teaches landscape architecture at the University of Illinois, Urbana-Champaign, also takes note of the social divisions among white South Africans. Far from appearing monolithic in their beliefs, they are shown to have exercised different attitudes toward their domestic servants based on their own experiences, which were rooted in differing ethics formed by class, ethnicity, language, religion, and gender. The homes and servant quarters described by the author manifest the different aspirations and anxieties of the women and men who inhabited them, reflecting the broader social conditions of a nation divided by race, by gender, and by class.

The difficulty of bridging two worlds—one rooted in the village life of largely Bantu-speaking people, the other a cosmopolitan realm of global commerce and culture—introduces and concludes At Home with Apartheid. Ginsburg notes the shock experienced by women who migrated from villages (or the black townships 

In [13]:
embeddings = FastEmbedEmbeddings(model_name = "BAAI/bge-large-en-v1.5")
qdrant = Qdrant.from_documents(documents,
                               embeddings,
                               path = '/content/drive/MyDrive/qdrant_db',
                               collection_name = 'History0fNepal_embeddings'
                               )

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/742 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

# perform dense retrieval using qdrant

In [14]:
%%time

query = "Who were Malla?"
similar_docs = qdrant.similarity_search_with_score(query)
dense_result = [(doc.page_content, score) for doc, score in similar_docs]

CPU times: user 793 ms, sys: 66 ms, total: 859 ms
Wall time: 49.9 ms


In [15]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()


text: 5 The Mallas of Jumla were not related to the better Preliminary known Regmi, Mallas of Kathmandu Valley. Cf. Nepal i. 713; Scientific Nepal Medieval History of Nepal (Calcutta, 1958), p. 81. Expeditions in 1965. Giuseppe (Rome and Tucci, 1951), p. Luciano

score: 0.6771418909676624
________________________________________________________________________________

text: CHAPTER VI

RAN MALLA, the second son of Rājā Yaksha Malla, became king of Banikāpur (Banēpa) and seven adjacent villages. He reigned 21 years, and died without issue. Dēvī gave a golden head of a buffalo to a certain oilman of Banikāpur, who, in consequen

score: 0.6478536150844008
________________________________________________________________________________

text: THE MALLA RĀJĀS OF KATHMANDU

the Manohara, they assaulted him with nōls and mudgars (sticks and clubs), and he was compelled to take refuge at Bhatgāon. The Rājā of Bhātgāon, knowing him to be a wicked man, kept him in confinement in one of the Chōk

In [16]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k": 5})
retrieved_docs = retriever.invoke(query)


CPU times: user 1.45 s, sys: 3.12 s, total: 4.56 s
Wall time: 59.5 ms


In [17]:

for doc in retrieved_docs:
  print(f"id: {doc.metadata['_id']}\n")
  print(f"text: {doc.page_content[:256]}")
  print("_" * 80)
  print()


id: 2fa4fc2a1b564fe1890e16e49ae2cdb5

text: 5 The Mallas of Jumla were not related to the better Preliminary known Regmi, Mallas of Kathmandu Valley. Cf. Nepal i. 713; Scientific Nepal Medieval History of Nepal (Calcutta, 1958), p. 81. Expeditions in 1965. Giuseppe (Rome and Tucci, 1951), p. Luciano
________________________________________________________________________________

id: d1213e32dc414da48f8e77e90f024450

text: CHAPTER VI

RAN MALLA, the second son of Rājā Yaksha Malla, became king of Banikāpur (Banēpa) and seven adjacent villages. He reigned 21 years, and died without issue. Dēvī gave a golden head of a buffalo to a certain oilman of Banikāpur, who, in consequen
________________________________________________________________________________

id: 2a2a87830f5f413185fd4056669a693c

text: THE MALLA RĀJĀS OF KATHMANDU

the Manohara, they assaulted him with nōls and mudgars (sticks and clubs), and he was compelled to take refuge at Bhatgāon. The Rājā of Bhātgāon, knowing him to

# perform sparse retrieval using BM25

In [18]:
from rank_bm25 import BM25Okapi

# Prepare texts for BM25 by tokenizing
texts = [doc.page_content for doc in documents]
tokenized_text = [text.split() for text in texts]
bm25 = BM25Okapi(tokenized_text)



In [19]:
def bm25_search(query, bm25, texts):
    # Tokenize the query and get BM25 scores for each document
    tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    # Combine scores with texts and sort them
    results = sorted(zip(scores, texts), reverse=True)
    return results

In [20]:
# Execute BM25 search
bm25_results = bm25_search(query, bm25, texts)

In [21]:
# Combine dense and BM25 results, ensuring scores are float
combined_results = [(text, float(score)) for text, score in dense_result] + [(text, float(score)) for score, text in bm25_results]

# Sort combined results by score (in descending order)
combined_results.sort(key=lambda x: x[1], reverse=True)

# Print the combined results
for text, score in combined_results:
    print(f"text: {text[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()


Streaming output truncated to the last 5000 lines.
score: 0.0
________________________________________________________________________________

text: Makunda Sen procured from the Nawab Vazir a grant of the extensive estate of Tilpur, and of that part of Rajpur, which is on the west side of the Gandaki, and had once, as lately mentioned, belonged to a branch of his family; but in the latter he never acq

score: 0.0
________________________________________________________________________________

text: Makunda Sen procured from the Nawab Vazir a grant of the extensive estate of Tilpur, and of that part of Rajpur, which is on the west side of the Gandaki, and had once, as lately mentioned, belonged to a branch of his family; but in the latter he never acq

score: 0.0
________________________________________________________________________________

text: Makunda Sen procured from the Nawab Vazir a grant of the extensive estate of Tilpur, and of that part of Rajpur, which is on the west si

In [22]:
# Rerank and compress
compressor = FlashrankRerank(model = 'ms-marco-MiniLM-L-12-v2')
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 59.2MiB/s]


In [23]:
%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

CPU times: user 0 ns, sys: 22 µs, total: 22 µs
Wall time: 42.9 µs
Running pairwise ranking..


3

In [24]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()


id: d1213e32dc414da48f8e77e90f024450

text: CHAPTER VI

RAN MALLA, the second son of Rājā Yaksha Malla, became king of Banikāpur (Banēpa) and seven adjacent villages. He reigned 21 years, and died without issue. Dēvī gave a golden head of a buffalo to a certain oilman of Banikāpur, who, in consequen

score: 0.9441864490509033
--------------------------------------------------------------------------------

id: 2fa4fc2a1b564fe1890e16e49ae2cdb5

text: 5 The Mallas of Jumla were not related to the better Preliminary known Regmi, Mallas of Kathmandu Valley. Cf. Nepal i. 713; Scientific Nepal Medieval History of Nepal (Calcutta, 1958), p. 81. Expeditions in 1965. Giuseppe (Rome and Tucci, 1951), p. Luciano

score: 0.9388393759727478
--------------------------------------------------------------------------------

id: 2a2a87830f5f413185fd4056669a693c

text: THE MALLA RĀJĀS OF KATHMANDU

the Manohara, they assaulted him with nōls and mudgars (sticks and clubs), and he was compelled to take re

In [25]:
# Initialize the ChatGroq model with specific parameters
llm = ChatGroq(temperature = 0.01, model = 'mixtral-8x7b-32768')


In [26]:
# Define a prompt template for the language model to use in generating answers.
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""


In [27]:
# Create a prompt template object with placeholders for context and question.
prompt = PromptTemplate(template = prompt_template,
                        input_variables = ['context' , 'question'] )


In [28]:
# Set up a RetrievalQA object for question answering

qa = RetrievalQA.from_chain_type(llm = llm,
                                chain_type = 'stuff',
                                retriever = compression_retriever,
                                chain_type_kwargs = {'prompt': prompt, "verbose": False})

In [29]:
%%time
response = qa.invoke("Where is Nepal?")


Running pairwise ranking..
CPU times: user 17.1 s, sys: 5.87 s, total: 23 s
Wall time: 953 ms


In [30]:

print_response(response)


Nepal is a country located in the Himalayas, in South Asia. In a strict sense, the name Nepal refers
to the Kathmandu valley, the cultural, administrative, and historic heart of the country. However,
it is commonly used to refer to the territory of the Gorkha Rajas, which spans about thirteen
degrees of longitude and five degrees of latitude. This territory includes regions to the east of
Kathmandu, such as the lands occupied by the Kirat Kichak tribe, who were once a powerful nation and
are frequently mentioned in Hindu legend.

In summary, Nepal is a country located in the Himalayas, and its capital and cultural heart is the
Kathmandu valley. The name Nepal is often used to refer to the larger territory of the Gorkha Rajas.


In [31]:
%%time
response = qa.invoke("Where did the name of Nepal came from?")

Running pairwise ranking..
CPU times: user 17.6 s, sys: 6.05 s, total: 23.7 s
Wall time: 817 ms


In [32]:

print_response(response)

The name "Nepal" is derived from the Sanskrit word "Nepala," which is said to have been the name of
a Muni (holy man) called Niyam. However, this etymology is considered a modern conceit by Colonel
Kirkpatrick, as both south and north Indian Brahmans agree in writing the name as "Nepal." The
Puranas, attributed to Vyas, also write the name as "Nepal." The country is commonly referred to as
"Niyampal" in Nepal, but this is likely a more recent development. It is important to note that the
name "Nepal" is often used to refer to the entire territory of the Gorkha Rajas, which is much
larger than the strict definition of Nepal Proper.


In [33]:
%%time
response = qa.invoke("How is prithvi Narayan Shah?")

Running pairwise ranking..
CPU times: user 16 s, sys: 5.82 s, total: 21.8 s
Wall time: 1.08 s


In [34]:

print_response(response)

Prithvi Narayan Shah was the tenth king of Gorkha who came to the throne in 1742. He inherited his
father's ambition to conquer the populous and fertile Kathmandu Valley. After several campaigns, he
completed the conquest of the valley in 1768 and established the Nepali capital at Kathmandu, where
it has been ever since. By 1769, he had taken over the area of modern Nepal.

Prithvi Narayan Shah was a man of decision and realism, who recognized the complexity of the task he
was undertaking. He was aware of the economic and military superiority of the Malla kings of the
Valley, but his ambition and determination led him to take the necessary steps to realize his
ambition. He was also pragmatic in his approach to Hinduism, adopting a more relaxed attitude and
incorporating elements of Gorakhnath's teachings and the Rajput nobles' influence.

These characteristics, along with the abundant supply of manpower, financial and economic security,
and the fighting men of the Magar, Khas, and Baga

In [35]:
%%time
response = qa.invoke("Who are Mallas?")

Running pairwise ranking..
CPU times: user 19 s, sys: 6.22 s, total: 25.2 s
Wall time: 8.39 s


In [36]:

print_response(response)

The Mallas were a group of rulers who established a kingdom in western Nepal in the 12th century.
They are not to be confused with the Regmi Mallas of the Kathmandu Valley. The Malla kingdom of
Jumla was formed by joining the kingdoms of Guge and Puranglo, and possibly Ladakh, and it reached
its peak during the reign of Prithvimalla in the 14th century. The kingdom was feudal in nature, and
when the central power weakened, the feudatory states became independent. The Mallas of Jumla ruled
over more than half of Greater Nepal for approximately two centuries. In central Nepal, the Sen
kings established a significant kingdom based in Makwanpur. The Sen kingdom included Jumla, which
was under the Mallas of Jumla.

Additional information:

* The Mallas of Jumla were known for their coinage, which was debased by the last Newari rulers,
causing a problem of exchange when Prithvinarayan Shah came to power in Kathmandu Valley.
* The Malla kingdom of Jumla was formed by invading Khasa tribes and

In [37]:
%%time
response = qa.invoke("Who was Jung Bahadur Rana?")


Running pairwise ranking..
CPU times: user 16.7 s, sys: 6.29 s, total: 22.9 s
Wall time: 26.4 s


In [38]:

print_response(response)

Jung Bahadur Rana, whose real name was Bir Narsingh Kunwar, was a prominent figure in the history of
Nepal and the Indian Mutiny. He was not of royal blood but rose to become prime minister of Nepal in
1842. After eliminating his major rivals and securing his power, he traveled to Britain and France
in 1850, despite prohibitions for high-caste Hindus. His journey exposed him to British pragmatism
and industrial might, convincing him that maintaining a relationship with the British was crucial
for Nepal's independence and his own power.

Jung Bahadur was also the head of the army and other departments, with his eldest brother serving as
Commander-in-Chief. His brothers and relatives held numerous high-ranking positions in the army. The
Nepalese army's uniforms varied, with silver badges on turbans as a distinctive mark, and the
artillery uniform being blue. The troops were equipped with a mix of old flint muskets, percussion
"brown Bess," and Enfield rifles, along with the traditional k

In [39]:
%%time
response = qa.invoke("Tell me something on King Birendra Bir Bikram Shah Dev?")


Running pairwise ranking..
CPU times: user 18.1 s, sys: 6.39 s, total: 24.4 s
Wall time: 20 s


In [40]:

print_response(response)

King Birendra Bir Bikram Shah Dev was the 12th king of the Shah Dynasty in Nepal, who ruled from
1972 to 2001. He was crowned at the age of 26, following the death of his father King Mahendra. King
Birendra is known for his efforts to establish Nepal as a democratic country. In 1990, he introduced
a new constitution that ended the partyless Panchayat system and established a multi-party system.
He was respected for his humility and down-to-earth personality. King Birendra, along with his
family, was tragically killed in a massacre in 2001.


In [41]:
%%time
response = qa.invoke("Tell me something on Shah Kings")


Running pairwise ranking..
CPU times: user 15.2 s, sys: 6.21 s, total: 21.5 s
Wall time: 18.8 s


In [42]:

print_response(response)

The Shah kings established the kingdom of Gorkha, according to Hamilton's account. Although there
are gaps in our knowledge about the origins of the Shah dynasty, Acharya's claim that Drabya Shah
created the kingdom "out of whole cloth" is unwarranted without evidence. The Shahs were likely of
the Suryabangsi (Surejbunsi) race and had Sanskrit names. It's worth noting that political alliances
in the region often differed from family alliances, as Hamilton points out.
